In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 2.6 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=beec03010c751632706b8c4fa604cb1fd3d9fff51c812d496eb78569e38c5641
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv   a0606.csv		 heatmap531.ipynb
 a0530.csv   a0607.csv		 Heat_Map_Online_language.ipynb
 a0531.csv   csv		 __MACOSX
 a0601.csv   csv.zip		 non_violant1.csv
 a0602.csv   dataframe0607.csv	 non_violant2.csv
 a0603.csv   dataframe_csv	 violant.csv
 a0604.csv   Github		'新建 Microsoft PowerPoint 演示文稿.pptx'
 a0605.csv   heatmap530.ipynb


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_608 = 'csv/US06_08.csv'

In [8]:
ct = pd.read_csv(ct_path_608)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,184096565021911_2947473805350826,Facebook,2020-06-08 21:09:26,2021-07-23 11:03:51,link,Portland police chief resigns amid George Floy...,abcnews.go.com,LONG POST - LOTS OF EMOTION - STRAIGHT FROM MY...,"Portland’s police chief resigned, just six mon...",1.0,https://abcn.ws/2YhQHQf,https://www.facebook.com/184096565021911/posts...,1434114,2.815068,1.0,213,77,73,22,8,6,11,1,0,1,62,9,33,13,3,14,4,8,0,3,19609,ABC News Politics,ABCNewsPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Expert political analysis From ABC News. Visit...,True,2012-01-03 16:24:17,en,0,19609|2947473805350826
1,295479040495545_3120680277975393,Facebook,2020-06-08 21:09:25,2021-08-06 02:02:09,link,Portland police chief resigns amid George Floy...,abcnews.go.com,LONG POST - LOTS OF EMOTION - STRAIGHT FROM MY...,"Portland’s police chief resigned, just six mon...",1.0,https://abcn.ws/2BNlj4z,https://www.facebook.com/295479040495545/posts...,321245,-1.815385,1.0,38,9,11,0,2,2,2,1,0,0,43,5,41,8,3,8,3,7,0,2,19601,This Week,ThisWeekABC,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,“This Week with George Stephanopoulos” is ABC ...,True,2011-12-20 21:44:38,en,0,19601|3120680277975393
2,168753769979032_1465789990275397,Facebook,2020-06-08 21:09:25,2021-07-09 21:55:56,youtube,John Boyega George Floyd protest London speech...,youtube.com,Subscribe to the Evening Standard on YouTube: ...,You might need a tissue.,2.0,https://www.youtube.com/watch?v=GGXEB25WdyQ,https://www.facebook.com/168753769979032/posts...,108012,-3.187500,2.0,20,7,0,5,0,0,0,0,0,1,24,17,9,8,6,8,12,18,0,4,18444,Girl Du Jour,GirlDuJour,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Kicking the cage since day one.\n\nI am neithe...,False,2013-08-14 01:41:02,en,0,18444|1465789990275397
3,111808396894398_283821683026401,Facebook,2020-06-08 21:09:00,2021-05-29 22:29:06,photo,John Boyega George Floyd protest London speech...,youtube.com,Subscribe to the Evening Standard on YouTube: ...,Statement regarding George Floyd and recent pr...,1.0,https://www.facebook.com/JovaniPatterson/photo...,https://www.facebook.com/111808396894398/posts...,0,1.105263,1.0,11,5,4,1,0,0,0,0,0,0,4,4,2,3,0,0,0,0,0,0,13711518,Jovani Patterson for Baltimore,JovaniPatterson,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Father, husband, community leader and candidat...",False,2019-10-07 14:05:05,en,0,13711518|283821683026401
4,1464455057166833_2662373337374993,Facebook,2020-06-08 21:08:57,2021-03-05 16:36:22,link,Commentary: The militarization of local police...,timesunion.com,"Last week, I joined thousands of peaceful prot...",Read County Executive Pat Ryan's op-ed in the ...,1.0,https://www.timesunion.com/opinion/article/Com...,https://www.facebook.com/1464455057166833/post...,709,-0.944444,1.0,0,0,0,1,0,0,0,0,0,0,7,2,2,3,1,0,1,2,0,0,8659935,Town of Rochester Democrats,TownofRochesterDemocrats,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,This page is published by the Town of Rocheste...,False,2014-09-13 13:28:11,en,0,8659935|2662373337374993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Portland': 1, 'Oregon': 1}",{'America': 1}
1,"{'Portland': 1, 'Oregon': 1}",{'America': 1}
2,{},{}
3,{},{}
4,"{'ULSTER COUNTY': 1, 'KINGSTON': 1, 'N.Y.': 1,...",{}


In [24]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,184096565021911_2947473805350826,Facebook,2020-06-08 21:09:26,2021-07-23 11:03:51,link,Portland police chief resigns amid George Floy...,abcnews.go.com,LONG POST - LOTS OF EMOTION - STRAIGHT FROM MY...,"Portland’s police chief resigned, just six mon...",1.0,https://abcn.ws/2YhQHQf,https://www.facebook.com/184096565021911/posts...,1434114,2.815068,1.0,213,77,73,22,8,6,11,1,0,1,62,9,33,13,3,14,4,8,0,3,19609,ABC News Politics,ABCNewsPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Expert political analysis From ABC News. Visit...,True,2012-01-03 16:24:17,en,0,19609|2947473805350826,"{'Portland': 1, 'Oregon': 1}",{'America': 1},{'Portland': 1}
1,295479040495545_3120680277975393,Facebook,2020-06-08 21:09:25,2021-08-06 02:02:09,link,Portland police chief resigns amid George Floy...,abcnews.go.com,LONG POST - LOTS OF EMOTION - STRAIGHT FROM MY...,"Portland’s police chief resigned, just six mon...",1.0,https://abcn.ws/2BNlj4z,https://www.facebook.com/295479040495545/posts...,321245,-1.815385,1.0,38,9,11,0,2,2,2,1,0,0,43,5,41,8,3,8,3,7,0,2,19601,This Week,ThisWeekABC,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,“This Week with George Stephanopoulos” is ABC ...,True,2011-12-20 21:44:38,en,0,19601|3120680277975393,"{'Portland': 1, 'Oregon': 1}",{'America': 1},{'Portland': 1}
2,168753769979032_1465789990275397,Facebook,2020-06-08 21:09:25,2021-07-09 21:55:56,youtube,John Boyega George Floyd protest London speech...,youtube.com,Subscribe to the Evening Standard on YouTube: ...,You might need a tissue.,2.0,https://www.youtube.com/watch?v=GGXEB25WdyQ,https://www.facebook.com/168753769979032/posts...,108012,-3.187500,2.0,20,7,0,5,0,0,0,0,0,1,24,17,9,8,6,8,12,18,0,4,18444,Girl Du Jour,GirlDuJour,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Kicking the cage since day one.\n\nI am neithe...,False,2013-08-14 01:41:02,en,0,18444|1465789990275397,{},{},{'London': 1}
3,111808396894398_283821683026401,Facebook,2020-06-08 21:09:00,2021-05-29 22:29:06,photo,John Boyega George Floyd protest London speech...,youtube.com,Subscribe to the Evening Standard on YouTube: ...,Statement regarding George Floyd and recent pr...,1.0,https://www.facebook.com/JovaniPatterson/photo...,https://www.facebook.com/111808396894398/posts...,0,1.105263,1.0,11,5,4,1,0,0,0,0,0,0,4,4,2,3,0,0,0,0,0,0,13711518,Jovani Patterson for Baltimore,JovaniPatterson,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Father, husband, community leader and candidat...",False,2019-10-07 14:05:05,en,0,13711518|283821683026401,{},{},{'London': 1}
4,1464455057166833_2662373337374993,Facebook,2020-06-08 21:08:57,2021-03-05 16:36:22,link,Commentary: The militarization of local police...,timesunion.com,"Last week, I joined thousands of peaceful prot...",Read County Executive Pat Ryan's op-ed in the ...,1.0,https://www.timesunion.com/opinion/article/Com...,https://www.facebook.com/1464455057166833/post...,709,-0.944444,1.0,0,0,0,1,0,0,0,0,0,0,7,2,2,3,1,0,1,2,0,0,8659935,Town of Rochester Democrats,TownofRochesterDemocrats,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,This page is published by the Town of Rocheste...,False,2014-09-13 13:28:11,en,0,8659935|2662373337374993,"{

In [25]:
ct.to_csv('a0608.csv')

In [26]:
ct=pd.read_csv('a0608.csv',lineterminator='\n')

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Portland': 1, 'Oregon': 1}",{'America': 1},{'Portland': 1}
1,"{'Portland': 1, 'Oregon': 1}",{'America': 1},{'Portland': 1}
2,{},{},{'London': 1}
3,{},{},{'London': 1}
4,"{'ULSTER COUNTY': 1, 'KINGSTON': 1, 'N.Y.': 1,...",{},{}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

0                                       [Portland, Oregon]
1                                       [Portland, Oregon]
2                                                       []
3                                                       []
4        [Mo., Iraq, Ulster County, N.Y., County, Fergu...
                               ...                        
14097                                                   []
14098                                                   []
14099                                             [Boston]
14100                                        [Minneapolis]
14101    [Los Angeles, Santa Monica Boulevard, California]
Name: NER-msg, Length: 9828, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

0                    [Portland, Oregon, America, Portland]
1                    [Portland, Oregon, America, Portland]
2                                                 [London]
3                                                 [London]
4        [Mo., Iraq, Ulster County, N.Y., County, Fergu...
                               ...                        
14097                                          [BLM, Iowa]
14098                                          [BLM, Iowa]
14099                                             [Boston]
14100                 [Minneapolis, Denver, DENVER, Colo.]
14101    [Los Angeles, Santa Monica Boulevard, Californ...
Name: NER, Length: 9828, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['portland', 'america', 'oregon']),
       list(['portland', 'america', 'oregon']), list(['london']), ...,
       list(['boston']),
       list(['denver', 'minneapolis', 'denver', 'colo.']),
       list(['los angeles', 'west hollywood', 'santa monica boulevard', 'california'])],
      dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['portland',
 'america',
 'oregon',
 'portland',
 'america',
 'oregon',
 'london',
 'london',
 'mo.',
 'iraq',
 'ulster county',
 'n.y.',
 'county',
 'ferguson',
 'ulster county',
 'kingston',
 'minneapolis',
 'pullman',
 'colorado',
 'philadelphia',
 'chicago',
 'chicago',
 'minneapolis',
 'us',
 'the united states',
 'minneapolis',
 'texas',
 'houston',
 'minneapolis',
 'nyc',
 'minneapolis',
 'minneapolis',
 'america',
 'vancouver',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'malibu',
 'california',
 'los angeles county',
 'agoura hills',
 'hollywood',
 'beverly hills',
 'west hollywood',
 'topanga',
 'santa monica',
 'brentwood',
 'sacramento',
 'bel air',
 'west los angeles',
 'pacific palisades',
 'alabama',
 'houston',
 'brooklyn',
 'brooklyn',
 'brooklyn',
 'minneapolis',
 'minnesota',
 'minn.',
 'u.s.',
 'minneapolis',
 'minneapolis',
 'utah',
 "new mexico's",
 "new mexico's",
 "new mexico's",
 'houston',
 "new mexico's",
 'the united states',
 'philando 

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('minneapolis', 3330),
 ('houston', 1748),
 ('america', 998),
 ('washington', 770),
 ('the united states', 582),
 ('texas', 514),
 ('us', 507),
 ('u.s.', 481),
 ('minnesota', 416),
 ('d.c.', 389),
 ('alabama', 303),
 ('california', 295),
 ('los angeles', 230),
 ('virginia', 225),
 ('seattle', 214),
 ('boston', 193),
 ('maryland', 192),
 ('canada', 187),
 ('buffalo', 185),
 ('france', 184),
 ('new york', 181),
 ('germany', 175),
 ('paris', 172),
 ('australia', 171),
 ('ireland', 162),
 ('berlin', 161),
 ('new york city', 161),
 ('mexico', 159),
 ('athens', 159),
 ('new zealand', 158),
 ('spain', 157),
 ('brazil', 156),
 ('dublin', 154),
 ('italy', 154),
 ('madrid', 154),
 ('ft.', 153),
 ('lauderdale', 153),
 ('barcelona', 152),
 ('brussels', 152),
 ('greece', 151),
 ('🏾new jersey’s', 151),
 ('amsterdam', 151),
 ('syria', 151),
 ('denmark', 150),
 ('copenhagen', 148),
 ('auckland', 144),
 ('mexico city', 144),
 ('poland', 142),
 ('iowa', 135),
 ('binnish', 133),
 ('philadelphia', 118),


In [43]:
dataframe608=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

1905

In [45]:
dataframe608.to_csv('dataframe0608.csv')